# Intro to the usage of the genericVAE
## Requirements

This notebook uses python >= 3.7 with tensorflow version 2.12.0, numpy version 1.23.5, pandas version 2.0.0 and h5py 3.8.0.

## Data

The GTEx V8 study serves as a brilliant dataset for demonstration.
At first we want to download a set that unifies all gene read counts, for different types of cells, in the study.

The inclusion and exclusion criteria were specified as:

1. 21 ≤ Age (years) ≤ 70
2. 18.5 < Body Mass Index < 35
3. Time between death and tissue collection less than 24 hours
4. No whole blood transfusion within 48 hours prior to death
5. No history of metastatic cancer
6. No chemotherapy or radiation therapy within the 2 years prior to death
7. Generally unselected for presence or absence of diseases or disorders, except for potentially communicable diseases that disqualify someone to donate organs or tissues would also be disqualifying for GTEx.

This data can be downloaded from the following URL: `https://storage.googleapis.com/gtex_analysis_v8/rna_seq_data/GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_reads.gct.gz` .
After the download, move it to a folder `genericVAE/data` where you'll place all other data, too.
To read the data a helping function is provided:

In [1]:
from helpers import reading
data = reading.read_gct_from_GTEx("data/GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_reads.gct.gz")
# data.info

We see, that there is rows specifying genes (by entrez-id and the HUGO-symbol) and columns specifying samples.
As the dataset is quite a chunk, for further analysis we want to safe this data in format, that's faster accessible, for us that's h5:

In [ ]:
data.to_hdf("data/GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_reads.h5", key="data")
del data

Next we want to filter the dataset for genes, that are relevant for our current scope.
For the beginning, lets focus on genes, that are part of the STRING database.
For this purpose there is a list provided, named `h_S_string.txt`.

In [36]:
import pandas as pd

In [37]:
data = pd.read_hdf("data/GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_reads.h5", key = "data")
in_string = pd.read_csv("h_S_string.txt", sep="\t")["preferred_name"]
# filter the frame
fdat = data.loc[data["Description"].isin(in_string)]

Now we have a subset of 17383 samples reduced to 18647 gene reads per sample.
Prior to training there needs to be done some pre-processing.
Experimentation has shown, that log-transformation and scaling is enough.
For the sake of performance we will switch to `numpy` and later reconstruct the data-frame structure.


In [3]:
import numpy as np
import helpers.data_preprocessing as prep

# for the log-transformation
logdat = prep.log_norm(np.array(fdat)[:, 2:])
# for the scaling, we scale sample-wise, here we add 1 to avoid zero-division:
sdat = prep.scale_by_sample(logdat+1)

## Build and fit a model

Now we can build a variational autoencoder (VAE) model to be trained with the data we just prepared.
At first we specify the parameters the model will be based on:

In [4]:
import class_definitions.generic_VAE as gvae

2023-04-17 13:36:18.764596: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-17 13:36:18.797184: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-17 13:36:18.797927: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-17 13:36:19.484279: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [5]:
# as we train sample-wise, we have to transpose the dataframe
input_shape = sdat.T.shape[1]

vae = gvae.Builder(input_shape=input_shape,
                   encoder_shape=[1000],
                   decoder_shape=[1000],
                   latent_dims=50)

vae = gvae.VAE(vae)

2023-04-17 13:36:20.066104: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


Next we compile it, in concordance with the `keras` functional API:

In [6]:
vae.compile()

Before we can start training, lets find a batch-size assuming we want to train in 100 epochs:

In [7]:
batchsize = sdat.shape[1]//100

And fit it to our pre-processed data:

In [29]:
vae.fit(sdat.T, epochs=10, batch_size= batchsize,workers=64, use_multiprocessing=True)

Epoch 1/10
101/101 [==============================] - 24s 238ms/step - loss: nan - reconstruction_loss: nan - kl_loss: nan
Epoch 2/10
101/101 [==============================] - 24s 236ms/step - loss: nan - reconstruction_loss: nan - kl_loss: nan
Epoch 3/10
101/101 [==============================] - 24s 236ms/step - loss: nan - reconstruction_loss: nan - kl_loss: nan
Epoch 4/10
101/101 [==============================] - 24s 235ms/step - loss: nan - reconstruction_loss: nan - kl_loss: nan
Epoch 5/10
101/101 [==============================] - 23s 232ms/step - loss: nan - reconstruction_loss: nan - kl_loss: nan
Epoch 6/10
101/101 [==============================] - 23s 231ms/step - loss: nan - reconstruction_loss: nan - kl_loss: nan
Epoch 7/10
101/101 [==============================] - 23s 231ms/step - loss: nan - reconstruction_loss: nan - kl_loss: nan
Epoch 8/10
101/101 [==============================] - 23s 232ms/step - loss: nan - reconstruction_loss: nan - kl_loss: nan
Epoch 9/10
101/1

In this case we just trained once with each sample.
So far no training best practice was established.

The difference here to usual cases of generative frameworks is, that the generative process is not the thing we aim to optimize, here the aim is to produce a representative latent space.

This difference could also allow the user to iterate several times over the same data and cause soft overfitting as we don't aim to create an intelligent model.
In case of using such a mechanism the introduction of drop-outs however is a good idea to still suppress overfitting effects getting too large.

In the end, the user has to set the hyper parameters in a way, that seems appropriate for the underlying case.

After fitting of the model one want's to extract the reconstruction errors.
These reconstruction errors are the basis for the further analysis.
The variational autoencoder reduces the input over our hidden layer towards
a representation of only 18 dimensions.
Therefor the system is ought to train itself in a way, that allows latent conservation of features that help to reduce this error.
Some features will contribute more to this latent representation, some contribute less.
Based on the contribution of a feature to the latent representation it's reconstruction error
will be smaller (high contribution) or greater (less contribution).
For this reason we'll remap the gene-names to the reconstruction errors:

In [30]:
# to extract reconstruction-errors we simply access the vae-obj
recons = vae.fwise_recon_error_tracker.result().numpy()
recon_frame = pd.DataFrame({"name" : fdat["Description"], "recon_1" : recons})

In [35]:
recon_frame

,name,recon_1
Name,,
ENSG00000186092.4,OR4F5,NaN
ENSG00000278566.1,OR4F29,NaN
ENSG00000273547.1,OR4F16,NaN
ENSG00000187634.11,SAMD11,NaN
ENSG00000188976.10,NOC2L,NaN
...,...,...
ENSG00000212907.2,MT-ND4L,NaN
ENSG00000198886.2,MT-ND4,NaN
ENSG00000198786.2,MT-ND5,NaN


## Analysis of the reconstruction error list

To dig a little deeper we use R.
To use R inside of this notebook we use the ipython rpy2 extension:

In [31]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


To invoke R in an IPythbon cell simply use the magic function `%%R`:

In [32]:
%%R 
string = paste("Hello, it's R ", R.Version()$major, R.Version()$minor, " printing this")
print(string)


[1] "Hello, it's R  4 2.3  printing this"


### Enrichment analysis

At first let's have a look the top-200 genes with the lowest reconstruction error and check the enriched terms:

In [33]:
%%R
library('tidyverse')
library('clusterProfiler')
library('org.Hs.eg.db')
library('tidyverse')
library('STRINGdb')

In [34]:
%%R -i recon_frame
str(recon_frame)

'data.frame':	18647 obs. of  2 variables:
 $ name   : chr  "OR4F5" "OR4F29" "OR4F16" "SAMD11" ...
 $ recon_1: num  NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN ...


In [25]:
%%R -i recon_frame

data = recon_frame %>% filter(recon_1 != 0)

ref = data$name

top_500 = data$name[1:501]

# for cellular component
ego = enrichGO(gene = top_500,
               OrgDb = org.Hs.eg.db,
               universe = ref,
               ont = "CC",
               keyType = "SYMBOL")

barplot(ego)

R[write to console]: --> No gene can be mapped....

R[write to console]: --> Expected input gene ID: TAF10,ERCC8,SIN3B,EPC1,ACTL6B,ZNF217

R[write to console]: --> return NULL...



NULL


In [ ]:
%%R
# and molecular function
ego = enrichGO(gene = top_500,
               OrgDb = org.Hs.eg.db,
               universe = ref,
               ont = "MF",
               keyType = "SYMBOL")

barplot(ego)

In [ ]:
%%R
# and biological process:
ego = enrichGO(gene = top_500,
               OrgDb = org.Hs.eg.db,
               universe = ref,
               ont = "BP",
               keyType = "SYMBOL")

barplot(ego)

### Network analysis

We can also look closer at the genes by plotting them as PPI network:

In [ ]:
%%R
string_db = STRINGdb$new(species=9606,
                         score_threshold=200,
                         input_directory="")

top_500 = data.frame("gene"=top_500)

top_500_mapped = string_db$map(top_500, "gene", removeUnmappedRows = T)
string_db$plot_network(top_500)

Seems like we have a few heavy interconnected regions.

In [ ]:
%%R
ntwrk <- string_db$get_subnetwork(top_500_mapped$STRING_id)

library('igraph')

With igraph, let us have a look at some measures we can easily compute on the extracted network:

In [ ]:
%%R
# 
order(betweenness(ntwrk))
mean_distance(ntwrk)
clique.number(ntwrk)

For a further analysis let's switch to the package `BioNAR`.
Let's have a look at how well our subnetwork follows the power-law:

In [ ]:
%%R
library('BioNAR')
clusters <- calcAllClustering(ntwrk)
pFit <- fitDegree( as.vector(igraph::degree(graph=clusters)),threads=1, Nsim=5,

                    plot=TRUE)

The package also allows us to identify clusters in the extracted list.
These clusters then can get extracted and be plotted clusterwise:

In [ ]:
%%R
alg = "louvain"
library("randomcoloR")

clsuters <- calcCentrality(ntwrk)
getCentralityMatrix(clusters)
clusters <- calcClustering(clsuters, alg)
summary(clusters)
V(clusters)$louvain

mem_df <- data.frame(names=V(clusters)$name,membership=as.numeric(V(clusters)$louvain))

palette <- distinctColorPalette(max(as.numeric(mem_df$membership)))

lay <- layoutByCluster(clusters, mem_df,layout = layout_nicely)

plot(clusters,vertex.size=3,layout=lay,
        vertex.label=NA,
        vertex.color=palette[as.numeric(mem_df$membership)],
        edge.color='grey95')

legend('topright',legend=names(table(mem_df$membership)),
        col=palette,pch=19,ncol = 2)

Or by just by their community interactions:

In [ ]:
%%R
idx <- base::match( V(clusters)$name, mem_df$names)

cgg <- getCommunityGraph(clusters, mem_df$membership[idx])

D0 = unname(degree(cgg))

plot(cgg, vertex.size=sqrt(V(cgg)$size), vertex.cex = 0.8, vertex.color=round(log(D0)) + 1, layout=layout_with_kk, margin=0)

## Transcription factor enrichment

As we now have a ranked list with "important" genes, we can check if it's significantly enriched in transcription-factors:

In [ ]:
from evaluation_methods.fisher_exact_for_gene_lists import f_exact_test
data = pd.read_csv("recons/GTEx_filtereded_with_genes_in_string.csv")
data = data.loc[data["recon_1"] != 0]
top_500 = data.iloc[0:500]
f_exact_test(set(top_500["name"]), set(data["name"]))

To get an idea, about how the p-value will change over the ordered list of errors, we simply:

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plist = []

for genes in range(0, 1300):
    top = data.iloc[0:genes]
    results = f_exact_test(set(top["name"]), set(data["name"]))
    plist.append(results[0][1])

In [ ]:
plt.plot(plist)

# Training with large files using a data generator

If we want to work with large datasets, the `DataGenerator`-class implements everything needed to read from an h5 file, for training directly.
This can come in handy for example when the RAM of the used computer is too small to load a whole set at once.
With the generator you're able to shuffle the data between the batches, that can be useful in case one wants to iterate over the same data multiple times.
The class has left some space for batch wise pre-processing.
In this case we use a log-normalized and scaled version of the `EBPlusPlusAdjustPANCAN_IlluminaHiSeq_RNASeqV2.geneExp.tsv` dataset from `https://gdc.cancer.gov/about-data/publications/pancanatlas`.
To use it with the data generator:

In [ ]:
import h5py
from class_definitions import generic_VAE
from class_definitions import data_handler
import numpy as np


# get information about the shape:
data = h5py.File("./data/EBPlusPlusAdjustPANCAN.h5").get("transposed_data")
# and define the generator
data_train = data_handler.DataGenerator(dataset_name="transposed_data", 
                                        filepath="./data/EBPlusPlusAdjustPANCAN.h5",
                                        # batch size hereby  
                                        batch_size=10,
                                        # shuffel the data
                                        shuffle=True)
input_dims = data[0].shape[0]

# build a VAE:
vae_build = generic_VAE.Builder(
        input_dims,
        # using a deeper model than before
        [10000, 1000, 500],
        [10000, 1000, 500],
        100,
        dropout_rate=.01)

# make it a Model:
vae_model = generic_VAE.VAE(vae_build)

# complile it:
vae_model.compile()

# for the amount of batches:
batch_size = int(np.floor(input_dims / 100))

# vae_model.fit(data, batch_size=batch_size)
vae_model.fit(data_train)

# Filters applied prior training

In the introduction we simply searched for all genes, that were present in the STRINGdb database.
This lead to a latent space, learned from the feature data.
However this data was filtered prior to training, therefor we introduced a bias.

Using this bias to focus on a particular group of genes can come in handy.
A GO-term mapping a certain process for example is a great filter as we can specify gene products with it, that are known to be a part of it.

For example counts the GO-term `GO:0006355` specifies the process of *regulation of DNA-templated transcription* and counts currently 21'789'333 annotations.
The term `GO:0008134` specifies *transcription factor binding* and counts 96'125 annotations.

## define a Filter

We can get the names of genes (for *Homo sapiens*) annotated to these terms by using R again:

In [ ]:
%%R -o trans_assoc_genes
library(GO.db)
library(org.Hs.eg.db)

# the ID's
go_id = "GO:0006355"
tf_bnd_id = "GO:0008134"

# to get the genes annotated to the terms above
allegs = get(go_id, org.Hs.egGO2ALLEGS)
genes = unlist(mget(allegs,org.Hs.egSYMBOL))

allegs = get(tf_bnd_id, org.Hs.egGO2ALLEGS)
tf_bnd_genes = unlist(mget(allegs, org.Hs.egSYMBOL))

trans_assoc_genes = c(tf_bnd_genes, genes)

To get arid of redundancies we'll simply make it a set:

In [ ]:
trans_assoc_genes = set(trans_assoc_genes)

## How to use the Filter

To apply this list of genes as a filter to the already used GTEx dataset we just:

In [ ]:
data = pd.read_hdf("data/GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_reads.h5", key = "data")
data = data.loc[data["Description"].isin(trans_assoc_genes)]

## Model fitting

This datset now can undergo some pre-processing, this time we will also remove genes that count less than 17383 reads per gene for all samples.
And in the next step we use it to fit a VAE-model:

In [ ]:
data = data.drop(data[data.iloc[:, 2:].sum(axis=1) < 17383].index)

fdat = np.array(data)

logdat = prep.log_norm(np.array(fdat)[:, 2:])

sdat = prep.scale_by_sample(logdat)

input_shape = sdat.T.shape[1]
vae = gvae.Builder(input_shape=input_shape,
                   encoder_shape=[186],
                   decoder_shape=[186],
                   latent_dims=18,
                   dropout_rate=.001)

vae = gvae.VAE(vae)

# compile it:
vae.compile()

# and train it:
vae.fit(sdat.T, epochs=100, workers=64, use_multiprocessing=True)

In [ ]:
recons = vae.fwise_recon_error_tracker.result().numpy()
recon_frame = pd.DataFrame({"name" : data["Description"], "recon_1" : recons}).sort_values(by="recon_1")

## Further analysis

Like we already did.

In [ ]:
%%R -i recon_frame
ref = recon_frame$name

top_500 = recon_frame$name[1:501]

# for cellular component
ego = enrichGO(gene = top_500,
               OrgDb = org.Hs.eg.db,
               universe = ref,
               ont = "CC",
               keyType = "SYMBOL")
barplot(ego)

In [ ]:
%%R
# for BP
ego = enrichGO(gene = top_500,
               OrgDb = org.Hs.eg.db,
               universe = ref,
               ont = "BP",
               keyType = "SYMBOL")
barplot(ego)

In [ ]:
%%R
# for MF
ego = enrichGO(gene = top_500,
               OrgDb = org.Hs.eg.db,
               universe = ref,
               ont = "MF",
               keyType = "SYMBOL")
barplot(ego)

In [ ]:
%%R
library('MOMA')

ekegg = enrichKEGG(gene = mapHugo(top_500),
                   organism="hsa")
barplot(ekegg)

For the computation of the p-values used in the fishers exact test, the defaulted list holds transcription factors from *Mus musculus* and *Homo sapiens*.
If one want's to focus on different sets, simply hand them directly:

In [ ]:
tfs_list = pd.read_csv("evaluation_methods/trrust_rawdata.human.tsv", sep="\t")
# remove redundancies:
tfs = set(tfs_list["AATF"])

In [ ]:
import evaluation_methods.fisher_exact_for_gene_lists as ftest
data = recon_frame.loc[recon_frame["recon_1"] != 0]
top_100 = recon_frame.iloc[0:100]
ftest.f_exact_test(set(top_100["name"]), set(recon_frame["name"]), tfs)

In [ ]:
top_500 = recon_frame.iloc[0:500]
ftest.f_exact_test(set(top_500["name"]), set(recon_frame["name"]), tfs)

plist = []

for genes in range(0, 1300):
    top = recon_frame.iloc[0:genes]
    results = ftest.f_exact_test(set(top["name"]), set(recon_frame["name"]), tfs)
    plist.append(results[0][1])

plt.plot(plist)

As we're interested in the tf's, that have the lowest reconstruction error, we need to get an idea, where our list needs to be cut off:

In [ ]:
idx = plist.index(min(plist))
top_of_the_pops = data[0:idx]

To get some more knowledge regarding the function of these transcription factors, lets extract them:

tfs_in_tops = tfs & set(top_of_the_pops)

# Comparing two similar sets

The data sets under `https://doi.org/10.1038/sdata.2018.61` supply us with data for cancer and normal tissue RNA-seq data.
From there we downloaded breast cancer data of normal tissue from and also tumor tissue.

In [ ]:
# the cancer data:
brca = pd.read_csv("/home/test/Downloads/brcarsemfpkmtcgat-tumor.txt.gz", sep="\t")
# the normal data from gtex:
gtex = pd.read_csv("/home/test/Downloads/breastrsemfpkmgtex.txt.gz", sep="\t")
# and the normal data from tcga
ntgc = pd.read_csv("/home/test/Downloads/brcarsemfpkmtcga.txt.gz", sep="\t")
# and join the normal data:
normal = pd.concat([gtex, ntgc]).fillna(0)

These now can be used for training VAEs, but at first the pre-processing takes place:

## On the normal data

In [ ]:
normal = normal.loc[normal["Hugo_Symbol"].isin(trans_assoc_genes)]
data = normal.drop(normal[normal.iloc[:, 2:].sum(axis=1) < 201].index)
fdat = np.array(data)

logdat = prep.log_norm(np.array(fdat)[:, 2:])

sdat = prep.scale_by_sample(logdat)

input_shape = sdat.T.shape[1]
vae = gvae.Builder(input_shape=input_shape,
                   encoder_shape=[500],
                   decoder_shape=[500],
                   latent_dims=100)

vae = gvae.VAE(vae)

# compile it:
vae.compile()

# and train it:
vae.fit(sdat.T, epochs=10, batch_size=200 , workers=64, use_multiprocessing=True)
# extract the recons:
recons = vae.fwise_recon_error_tracker.result().numpy()
recon_frame = pd.DataFrame({"name" : data["Hugo_Symbol"], "recon_1" : recons}).sort_values(by="recon_1")

In [ ]:

recon_frame

In [ ]:
%%R -i recon_frame

data = recon_frame %>% filter(recon_1 != 0)

ref = data$name

top_500 = data$name[1:501]

# for cellular component
ego = enrichGO(gene = top_500,
               OrgDb = org.Hs.eg.db,
               universe = ref,
               ont = "BP",
               keyType = "SYMBOL")

barplot(ego)

## Cancer-data

In [ ]:
tumor = brca.loc[brca["Hugo_Symbol"].isin(trans_assoc_genes)]
data = tumor.fillna(0)
data = tumor.drop(tumor[tumor.iloc[:, 2:].sum(axis=1) < 201].index)
fdat = np.array(data)

logdat = prep.log_norm(np.array(fdat)[:, 2:])

sdat = prep.scale_by_sample(logdat)

input_shape = sdat.T.shape[1]
vae = gvae.Builder(input_shape=input_shape,
                   encoder_shape=[500],
                   decoder_shape=[500],
                   latent_dims=100)

vae = gvae.VAE(vae)

# compile it:
vae.compile()

# and train it:
vae.fit(sdat.T, epochs=40, workers=64, use_multiprocessing=True)
# extract the recons:
recons = vae.fwise_recon_error_tracker.result().numpy()
recon_frame = pd.DataFrame({"name" : data["Hugo_Symbol"], "recon_1" : recons}).sort_values(by="recon_1")

In [ ]:
%%R -i recon_frame

data = recon_frame %>% filter(recon_1 != 0)

ref = data$name

top_500 = data$name[1:501]

# for cellular component
ego = enrichGO(gene = top_500,
               OrgDb = org.Hs.eg.db,
               universe = ref,
               ont = "BP",
               keyType = "SYMBOL")

barplot(ego)

# VAE-Architecture

Most of the scenarios above were optimized with interactive processes.
The architecture in the beginning was kept as shallow as possible.
As a rough guideline one can reduce the amounts of neurons at the beginning with 2-3 times the power of ten and create a latent layer of a size between 100 and 10 neurons.
If this does not lead to a sufficient outcome under a training considering all sample-sets once, the VAE can usually achieve better results by adding another hidden layer.
It's important to notice, that this can have an impact to the final ranking of genes by their reconstruction errors.